In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd

**Getting the soup**

In [2]:
html_text = requests.get('https://www.iplt20.com/auction').text
# print(html_text)
soup = BeautifulSoup(html_text, 'lxml')

**Getting the urls for every year and making it into a dict**

In [3]:
years = soup.find('ul' ,class_ ="year-auction nav").find_all('li')
print(years)
# years_list = years.findall('li')
# print(years_list)

[<li class="active"><a class="active show" href="https://www.iplt20.com/auction/2023">2023</a></li>, <li class=""><a class="" href="https://www.iplt20.com/auction/2022">2022</a></li>, <li class=""><a class="" href="https://www.iplt20.com/auction/2021">2021</a></li>, <li class=""><a class="" href="https://www.iplt20.com/auction/2020">2020</a></li>, <li class=""><a class="" href="https://www.iplt20.com/auction/2019">2019</a></li>, <li class=""><a class="" href="https://www.iplt20.com/auction/2018">2018</a></li>, <li class=""><a class="" href="https://www.iplt20.com/auction/2017">2017</a></li>, <li class=""><a class="" href="https://www.iplt20.com/auction/2016">2016</a></li>, <li class=""><a class="" href="https://www.iplt20.com/auction/2015">2015</a></li>, <li class=""><a class="" href="https://www.iplt20.com/auction/2014">2014</a></li>, <li class=""><a class="" href="https://www.iplt20.com/auction/2013">2013</a></li>]


In [4]:
years_dict = {}
for year in years:
    years_dict[year.text] = year.a['href']
    
print(years_dict)

{'2023': 'https://www.iplt20.com/auction/2023', '2022': 'https://www.iplt20.com/auction/2022', '2021': 'https://www.iplt20.com/auction/2021', '2020': 'https://www.iplt20.com/auction/2020', '2019': 'https://www.iplt20.com/auction/2019', '2018': 'https://www.iplt20.com/auction/2018', '2017': 'https://www.iplt20.com/auction/2017', '2016': 'https://www.iplt20.com/auction/2016', '2015': 'https://www.iplt20.com/auction/2015', '2014': 'https://www.iplt20.com/auction/2014', '2013': 'https://www.iplt20.com/auction/2013'}


**Creating soup for player and team tables for all years. Making dicts for both respectively and building dataframe out of it.**

In [5]:
# original

num_team = 0
team_dict = {}
num_player = 0
player_dict = {}

for i in years_dict:
    season_url = years_dict[i]
    season = i
    
    season_html = requests.get(season_url).text
    season_soup = BeautifulSoup(season_html, 'lxml')
    
    team_overviews = season_soup.find('div', class_="ih-pt-tab-bg")
    team_overview = team_overviews.find_all('tr')

    
    for team_ov in team_overview[1:]:
        details = team_ov.find_all('td')
        
        team_name = details[0].text.strip()
        remaining_fund = details[1].text.strip()
        num_overseas_player = details[2].text.strip()
        total_players = details[3].text.strip()
        
        num_team = num_team + 1
        team_dict[num_team] =[season,team_name,remaining_fund,num_overseas_player,total_players]
        
    
    
   
    p = season_soup.find('section', class_ ="ih-points-table-sec").parent
    players = p.find_all('div', class_ ="ih-points-table-sec")

    for player in players:
        
        team_namee = player.h2.text.strip()

        player_details = player.find_all('tr')
        
        for player_detail in player_details[1:]:
            pl_detail = player_detail.text
            if season in ['2023','2022']:
                player_name = pl_detail.split('\n')[1].strip()
                nationality = pl_detail.split('\n')[2].strip()
                type = pl_detail.split('\n')[3].strip()
                price_paid = pl_detail.split('\n')[4].strip()
            else :
                player_name = pl_detail.split('\n')[1].strip()
                type = pl_detail.split('\n')[2].strip()
                price_paid = pl_detail.split('\n')[3].strip()
                nationality = np.nan
            
            num_player += 1
            player_dict[num_player] = [season,team_namee,player_name,nationality,type,price_paid]
        

In [6]:
cols_player = ['season','team_name','player_name','nationality','type','price_paid']
player_df = pd.DataFrame.from_dict(player_dict, orient = 'index', columns = cols_player)

cols_team = ['season','team_name','remaining_fund','overseas_player','total_players']
team_df = pd.DataFrame.from_dict(team_dict, orient = 'index', columns = cols_team)

# **team_df cleaning and transformation**

In [7]:
team_df

,season,team_name,remaining_fund,overseas_player,total_players
1,2023,Chennai Super Kings,"₹1,50,00,000",8,25
2,2023,Delhi Capitals,"₹4,45,00,000",8,25
3,2023,Gujarat Titans,"₹4,45,00,000",8,25
4,2023,Kolkata Knight Riders,"₹1,65,00,000",8,22
5,2023,Lucknow Super Giants,"₹3,55,00,000",8,25
...,...,...,...,...,...
89,2013,Mumbai Indians,"$341,739",11,28
90,2013,Pune Warriors India,"$804,130",11,33
91,2013,Rajasthan Royals,"$7,071,304",10,21
92,2013,Royal Challengers Bangalore,"$711,522",10,31


In [8]:
team_df.describe()

,season,team_name,remaining_fund,overseas_player,total_players
count,93,93,93,93,93
unique,11,15,77,5,11
top,2023,Kolkata Knight Riders,₹0,8,25
freq,10,11,4,55,35


In [9]:
for cols in team_df:
    print(team_df[cols].unique())

['2023' '2022' '2021' '2020' '2019' '2018' '2017' '2016' '2015' '2014'
 '2013']
['Chennai Super Kings' 'Delhi Capitals' 'Gujarat Titans'
 'Kolkata Knight Riders' 'Lucknow Super Giants' 'Mumbai Indians'
 'Punjab Kings' 'Rajasthan Royals' 'Royal Challengers Bangalore'
 'Sunrisers Hyderabad' 'Kings XI Punjab' 'Delhi Daredevils'
 'Gujarat Lions' 'Rising Pune Supergiant' 'Pune Warriors India']
['₹1,50,00,000' '₹4,45,00,000' '₹1,65,00,000' '₹3,55,00,000' '₹5,00,000'
 '₹12,20,00,000' '₹3,35,00,000' '₹1,75,00,000' '₹6,55,00,000'
 '₹2,95,00,000' '₹10,00,000' '₹15,00,000' '₹45,00,000' '₹0' '₹3,45,00,000'
 '₹95,00,000' '₹1,55,00,000' '₹2,55,00,000' '₹2,15,00,000' '₹3,20,00,000'
 '₹3,65,00,000' '₹18,80,00,000' '₹13,65,00,000' '₹35,00,000'
 '₹6,95,00,000' '₹9,00,00,000' '₹16,50,00,000' '₹8,50,00,000'
 '₹1,95,00,000' '₹14,75,00,000' '₹6,40,00,000' '₹10,10,00,000'
 '₹7,70,00,000' '₹3,70,00,000' '₹6,05,00,000' '₹7,15,00,000'
 '₹1,80,00,000' '₹5,30,00,000' '₹6,50,00,000' '₹1,60,00,000' '₹65,00,000'
 '₹

In [10]:
team_df.loc[team_df['team_name'] == 'Kings XI Punjab','team_name'] = 'Punjab Kings'
team_df.loc[team_df['team_name'] == 'Delhi Daredevils','team_name'] = 'Delhi Capitals'


In [11]:
team_df.remaining_fund = team_df['remaining_fund'].apply(lambda x : str(x).replace(',',''))
team_df.remaining_fund = team_df['remaining_fund'].apply(lambda x : str(x).replace('$',''))
team_df.remaining_fund = team_df['remaining_fund'].apply(lambda x : str(x).replace('₹','')).astype('int')

In [12]:
# 58.5501 was roughly the exchage rate for dollar to rupee in 2013
team_df.loc[team_df['season']=='2013','remaining_fund'] =  team_df.loc[team_df['season']=='2013','remaining_fund'] * 58.5501

In [13]:
team_df.rename(columns = {'remaining_fund':'remaining_fund(in rupee)'}, inplace = True)

In [14]:
team_df = team_df.astype({'overseas_player':int, 'total_players':int})


In [15]:
team_df.dtypes

season                       object
team_name                    object
remaining_fund(in rupee)    float64
overseas_player               int64
total_players                 int64
dtype: object

In [16]:
team_df.duplicated().sum()

0

In [17]:
for col in team_df:
    print(col, ':', team_df[col].isnull().sum())

season : 0
team_name : 0
remaining_fund(in rupee) : 0
overseas_player : 0
total_players : 0


In [18]:
team_df.reset_index(inplace = True)

In [19]:
team_df.rename(columns = {'index':'id'}, inplace = True)

In [20]:
team_df

,id,season,team_name,remaining_fund(in rupee),overseas_player,total_players
0,1,2023,Chennai Super Kings,1.500000e+07,8,25
1,2,2023,Delhi Capitals,4.450000e+07,8,25
2,3,2023,Gujarat Titans,4.450000e+07,8,25
3,4,2023,Kolkata Knight Riders,1.650000e+07,8,22
4,5,2023,Lucknow Super Giants,3.550000e+07,8,25
...,...,...,...,...,...,...
88,89,2013,Mumbai Indians,2.000885e+07,11,28
89,90,2013,Pune Warriors India,4.708189e+07,11,33
90,91,2013,Rajasthan Royals,4.140256e+08,10,21
91,92,2013,Royal Challengers Bangalore,4.165968e+07,10,31


In [21]:
team_df.to_csv('ipl_team_13-23.csv', index = False)

# **player_df cleaning and transformation**

In [22]:
player_df

,season,team_name,player_name,nationality,type,price_paid
1,2023,Chennai Super Kings,Ajinkya Rahane,Indian,Batter,"50,00,000"
2,2023,Chennai Super Kings,Bhagath Varma,Indian,All-Rounder,"20,00,000"
3,2023,Chennai Super Kings,Kyle Jamieson,Overseas,Bowler,"1,00,00,000"
4,2023,Chennai Super Kings,Ajay Mandal,Indian,All-Rounder,"20,00,000"
5,2023,Chennai Super Kings,Nishant Sindhu,Indian,All-Rounder,"60,00,000"
...,...,...,...,...,...,...
1046,2013,Sunrisers Hyderabad,Darren Sammy,NaN,All-Rounder,"425,000"
1047,2013,Sunrisers Hyderabad,Sudeep Tyagi,NaN,Bowler,"100,000"
1048,2013,Sunrisers Hyderabad,Clinton McKay,NaN,Bowler,"100,000"
1049,2013,Sunrisers Hyderabad,Nathan McCullum,NaN,Bowler,"100,000"


In [23]:
player_df.describe()

,season,team_name,player_name,nationality,type,price_paid
count,1050,1050,1050,284,1050,1050
unique,11,15,603,2,6,123
top,2022,Sunrisers Hyderabad,Jaydev Unadkat,Indian,All-Rounder,"20,00,000"
freq,204,125,10,188,380,198


In [24]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1050 entries, 1 to 1050
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   season       1050 non-null   object
 1   team_name    1050 non-null   object
 2   player_name  1050 non-null   object
 3   nationality  284 non-null    object
 4   type         1050 non-null   object
 5   price_paid   1050 non-null   object
dtypes: object(6)
memory usage: 57.4+ KB


In [25]:
for cols in player_df:
    print(player_df[cols].unique())

['2023' '2022' '2021' '2020' '2019' '2018' '2017' '2016' '2015' '2014'
 '2013']
['Chennai Super Kings' 'Delhi Capitals' 'Gujarat Titans'
 'Kolkata Knight Riders' 'Lucknow Super Giants' 'Mumbai Indians'
 'Punjab Kings' 'Rajasthan Royals' 'Royal Challengers Bangalore'
 'Sunrisers Hyderabad' 'Kings XI Punjab' 'Delhi Daredevils'
 'Gujarat Lions' 'Rising Pune Supergiant' 'Pune Warriors India']
['Ajinkya Rahane' 'Bhagath Varma' 'Kyle Jamieson' 'Ajay Mandal'
 'Nishant Sindhu' 'Shaik Rasheed' 'Ben Stokes' 'Phil Salt' 'Ishant Sharma'
 'Rilee Rossouw' 'Manish Pandey' 'Mukesh Kumar' 'Shivam Mavi'
 'K.S. Bharat' 'Urvil Patel' 'Mohit Sharma' 'Joshua Little'
 'Kane Williamson' 'Odean Smith' 'Kulwant Khejroliya' 'Shakib Al Hasan'
 'Vaibhav Arora' 'N. Jagadeesan' 'David Wiese' 'Mandeep Singh'
 'Litton Das' 'Suyash Sharma' 'Amit Mishra' 'Nicholas Pooran'
 'Romario Shepherd' 'Daniel Sams' 'Swapnil Singh' 'Yudhvir Charak'
 'Prerak Mankad' 'Yash Thakur' 'Naveen Ul Haq' 'Jaydev Unadkat'
 'Jhye Richardson' 

In [26]:
player_df.loc[player_df['team_name'] == 'Kings XI Punjab','team_name'] = 'Punjab Kings'
player_df.loc[player_df['team_name'] == 'Delhi Daredevils','team_name'] = 'Delhi Capitals'


In [27]:
player_df.loc[player_df['type'] == 'Batter','type'] = 'Batsman'
player_df.loc[player_df['type'] == 'Wicket Keeper','type'] = 'Wicket-Keeper'

In [28]:
player_df.price_paid = player_df['price_paid'].apply(lambda x : str(x).replace(',','')).astype(int)

In [29]:
# 58.5501 was roughly the exchage for dollar to rupee in 2013
player_df.loc[player_df['season']=='2013','price_paid'] =  player_df.loc[player_df['season']=='2013','price_paid'] * 58.5501

In [30]:
player_df.rename(columns = {'price_paid':'price_paid(in rupee)'}, inplace = True)

In [31]:
for col in player_df:
    print(col, ":", player_df[col].isnull().sum())

season : 0
team_name : 0
player_name : 0
nationality : 766
type : 0
price_paid(in rupee) : 0


In [32]:
player_df.duplicated().sum()

0

In [33]:
player_df.loc[(player_df.nationality.isnull())  & (player_df.player_name.duplicated()), : ]

,season,team_name,player_name,nationality,type,price_paid(in rupee)
288,2021,Chennai Super Kings,K.Bhagath Varma,NaN,All-Rounder,2000000.0
293,2021,Delhi Capitals,Sam Billings,NaN,Wicket-Keeper,20000000.0
294,2021,Delhi Capitals,Umesh Yadav,NaN,Bowler,10000000.0
295,2021,Delhi Capitals,Ripal Patel,NaN,All-Rounder,2000000.0
296,2021,Delhi Capitals,Vishnu Vinod,NaN,Wicket-Keeper,2000000.0
...,...,...,...,...,...,...
1041,2013,Royal Challengers Bangalore,Ravi Rampaul,NaN,Bowler,16979529.0
1043,2013,Royal Challengers Bangalore,Daniel Christian,NaN,All-Rounder,5855010.0
1045,2013,Sunrisers Hyderabad,Thisara Perera,NaN,All-Rounder,39521317.5
1046,2013,Sunrisers Hyderabad,Darren Sammy,NaN,All-Rounder,24883792.5


In [34]:
df = player_df.copy()

In [35]:
df.drop_duplicates('player_name', keep = 'first', inplace = True)

In [36]:
df1 = df.loc[:,['player_name','nationality']].copy()

In [37]:
df1['nationality'].isnull().sum()

337

In [38]:
df1_dict = df1.set_index('player_name').to_dict('index')

In [39]:
df1_dict['Ajinkya Rahane']['nationality']

'Indian'

In [40]:
for i,r in player_df.iterrows():
    if r.player_name in df1_dict:
        player_df.loc[i,'nationality'] = df1_dict[r.player_name]['nationality']

In [41]:
for col in player_df:
    print(col, ":", player_df[col].isnull().sum())

season : 0
team_name : 0
player_name : 0
nationality : 533
type : 0
price_paid(in rupee) : 0


In [42]:
player_df.reset_index(inplace = True)

In [43]:
player_df.rename(columns = {'index':'id'}, inplace = True)

In [44]:
player_df

,id,season,team_name,player_name,nationality,type,price_paid(in rupee)
0,1,2023,Chennai Super Kings,Ajinkya Rahane,Indian,Batsman,5000000.0
1,2,2023,Chennai Super Kings,Bhagath Varma,Indian,All-Rounder,2000000.0
2,3,2023,Chennai Super Kings,Kyle Jamieson,Overseas,Bowler,10000000.0
3,4,2023,Chennai Super Kings,Ajay Mandal,Indian,All-Rounder,2000000.0
4,5,2023,Chennai Super Kings,Nishant Sindhu,Indian,All-Rounder,6000000.0
...,...,...,...,...,...,...,...
1045,1046,2013,Sunrisers Hyderabad,Darren Sammy,NaN,All-Rounder,24883792.5
1046,1047,2013,Sunrisers Hyderabad,Sudeep Tyagi,NaN,Bowler,5855010.0
1047,1048,2013,Sunrisers Hyderabad,Clinton McKay,NaN,Bowler,5855010.0
1048,1049,2013,Sunrisers Hyderabad,Nathan McCullum,NaN,Bowler,5855010.0


In [45]:
team_df.season = team_df.season.astype('str')
player_df.season = player_df.season.astype('str')


df_team = team_df.loc[:,['id','season','team_name']].copy()

# print(df)

df_dict = df_team.set_index('id').T.to_dict('list')

# print(df_dict)

for i,row in player_df.iterrows():
    for k in df_dict:
        if (df_dict[k][0] == row.season) and (df_dict[k][1] == row.team_name):
            player_df.loc[i, 'id'] = k
            break

print(player_df.dtypes)

id                        int64
season                   object
team_name                object
player_name              object
nationality              object
type                     object
price_paid(in rupee)    float64
dtype: object


In [46]:
player_df

,id,season,team_name,player_name,nationality,type,price_paid(in rupee)
0,1,2023,Chennai Super Kings,Ajinkya Rahane,Indian,Batsman,5000000.0
1,1,2023,Chennai Super Kings,Bhagath Varma,Indian,All-Rounder,2000000.0
2,1,2023,Chennai Super Kings,Kyle Jamieson,Overseas,Bowler,10000000.0
3,1,2023,Chennai Super Kings,Ajay Mandal,Indian,All-Rounder,2000000.0
4,1,2023,Chennai Super Kings,Nishant Sindhu,Indian,All-Rounder,6000000.0
...,...,...,...,...,...,...,...
1045,93,2013,Sunrisers Hyderabad,Darren Sammy,NaN,All-Rounder,24883792.5
1046,93,2013,Sunrisers Hyderabad,Sudeep Tyagi,NaN,Bowler,5855010.0
1047,93,2013,Sunrisers Hyderabad,Clinton McKay,NaN,Bowler,5855010.0
1048,93,2013,Sunrisers Hyderabad,Nathan McCullum,NaN,Bowler,5855010.0


In [47]:
player_df.to_csv('ipl_player_13-23.csv', index = False)